In [11]:
import os
import laspy
import numpy as np
from osgeo import gdal, osr, gdal_array

def convert_las_to_tif(las_directory, grid_size_meters=5):
    for file in os.listdir(las_directory):
        if file.endswith('.las'):
            las_file = os.path.join(las_directory, file)
            tif_file = os.path.join(las_directory, os.path.splitext(file)[0] + '.tif')
            print(file)

            # Read LAS file
            las = laspy.read(las_file)

            # Convert grid size to feet
            grid_size_feet = grid_size_meters * 3.28084 

            # Extract coordinates and create grid
            min_x, max_x = np.floor(np.min(las.x)), np.ceil(np.max(las.x))
            min_y, max_y = np.floor(np.min(las.y)), np.ceil(np.max(las.y))
            x_grid = np.arange(min_x, max_x, grid_size_feet)
            y_grid = np.arange(min_y, max_y, grid_size_feet)
            grid_x, grid_y = np.meshgrid(x_grid, y_grid)

            # Digitize points
            x_idx = np.digitize(las.x, x_grid) - 1
            y_idx = np.digitize(las.y, y_grid) - 1

            # Create DEM
            dem = np.full(grid_x.shape, np.nan)
            dem[y_idx, x_idx] = las.z * 3.28084  # Convert from meters to feet
            dem = np.flipud(dem)

            # Fill no data values
            dem = np.where(np.isnan(dem), np.nanmin(dem), dem)

            # Create GDAL dataset
            driver = gdal.GetDriverByName('GTiff')
            out_raster = driver.Create(tif_file, dem.shape[1], dem.shape[0], 1, gdal.GDT_Float32)
            out_raster.SetGeoTransform((min_x, grid_size_feet, 0, max_y, 0, -grid_size_feet))
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(26915)
            out_raster.SetProjection(srs.ExportToWkt())

            # Write data
            out_band = out_raster.GetRasterBand(1)
            out_band.WriteArray(dem)
            out_band.SetNoDataValue(np.nan)

            # Close file
            out_raster = None

# Usage
las_directory = '/Volumes/CedarLakes/Cedar Lakes Data/TGLO/Lidar_Request'
convert_las_to_tif(las_directory)
